# Deploy credit risk classification model in Azure Container Instance (ACI)

Your model will be deployed as a web service in [Azure Container Instances](https://docs.microsoft.com/azure/container-instances/) (ACI). A web service is an image, in this case a Docker image, that encapsulates the scoring logic and the model itself. 

You will learn how to deploy model on Azure Machine Learning service:

 - Get and test the sample model locally
 - Register the model to your Azure workspace
 - Deploy the model to ACI
 - Test the deployed model

**Note:** ACI is a great solution for testing and understanding the workflow. For scalable production deployments, consider using Azure Kubernetes Service. For more information, see [how to deploy and where](https://docs.microsoft.com/azure/machine-learning/service/how-to-deploy-and-where).

Sample model was created using scikit-learn version: `0.20.2`. Model predict if there is a risk for credit.

## Install required python packages

In [1]:
!pip install --upgrade scikit-learn==0.20.2 | tail -n 1
!pip install --upgrade azureml-core  | tail -n 1

tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
azureml-core 1.0.45 has requirement ruamel.yaml<=0.15.89,>=0.15.35, but you'll have ruamel-yaml 0.11.14 which is incompatible.
Requirement not upgraded as not directly required: scipy>=0.13.3 in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from scikit-learn==0.20.2) (1.0.0)
tensorflow 1.3.0 requires tensorflow-tensorboard<0.2.0,>=0.1.0, which is not installed.
azureml-core 1.0.45 has requirement ruamel.yaml<=0.15.89,>=0.15.35, but you'll have ruamel-yaml 0.11.14 which is incompatible.
Requirement not upgraded as not directly required: pycparser in /opt/conda/envs/DSX-Python35/lib/python3.5/site-packages (from cffi!=1.11.3,>=1.8->cryptography!=1.9,!=2.0.*,!=2.1.*,!=2.2.*->azureml-core) (2.18)


### Action: Restart the kernel

## Get sample model

In [18]:
!rm -rf german_credit_risk.joblib
!wget https://github.com/pmservice/ai-openscale-tutorials/raw/master/applications/custom-ml-engine-bluemix/models/credit/german_credit_risk.joblib

--2019-06-27 10:34:50--  https://github.com/pmservice/ai-openscale-tutorials/raw/master/applications/custom-ml-engine-bluemix/models/credit/german_credit_risk.joblib
Resolving github.com (github.com)... 192.30.253.113
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/applications/custom-ml-engine-bluemix/models/credit/german_credit_risk.joblib [following]
--2019-06-27 10:34:50--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/applications/custom-ml-engine-bluemix/models/credit/german_credit_risk.joblib
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9183 (9.0K) [application/octet-stream]
Saving to: ‘german_credit_risk.jo

### Load model from local file

In [ ]:
from sklearn.externals import joblib

In [77]:
clf = joblib.load(os.path.join(os.getcwd(), model_path))

## Test model locally

In [78]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

### Load test data

In [79]:
!rm -rf credit_risk_training.csv
!wget https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/notebooks/data/credit_risk_training.csv

--2019-06-27 12:41:48--  https://raw.githubusercontent.com/pmservice/ai-openscale-tutorials/master/notebooks/data/credit_risk_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 694222 (678K) [text/plain]
Saving to: ‘credit_risk_training.csv’

100%[======================================>] 694,222     --.-K/s   in 0.02s   

2019-06-27 12:41:48 (28.1 MB/s) - ‘credit_risk_training.csv’ saved [694222/694222]



In [80]:
data_df = pd.read_csv("credit_risk_training.csv",
                      dtype={'LoanDuration': int, 
                             'LoanAmount': int, 
                             'InstallmentPercent': int, 
                             'CurrentResidenceDuration': int, 
                             'Age': int, 
                             'ExistingCreditsCount': int, 
                             'Dependents': int})
test_data = data_df
test_data = test_data.drop('Risk', axis=1)

### Predict on test data

In [86]:
predictions = clf['postprocessing'](clf['model'].predict(test_data))

print("Model accuracy: {:.3f}".format(accuracy_score(data_df.Risk, predictions)))

Model accuracy: 0.778


## Register model

In [ ]:
import os
from azureml.core import Workspace
from azureml.core.model import Model
from azureml.core.authentication import InteractiveLoginAuthentication

In [19]:
model_name = "german_credit_risk"
model_path = "german_credit_risk.joblib"

Please provide your Azure ML Service credentials.

In [87]:
az_ml_service_credentials = {'tenant_id': '***',
                             'subscription_id': '***', 
                             'resource_group': '***',
                             'workspace_name': '***'}

In [43]:
# The code was removed by Watson Studio for sharing.

In [17]:
interactive_auth = InteractiveLoginAuthentication(tenant_id=az_ml_service_credentials['tenant_id'])

ws = Workspace(subscription_id=az_ml_service_credentials['subscription_id'], 
               resource_group=az_ml_service_credentials['resource_group'], 
               workspace_name=az_ml_service_credentials['workspace_name'], 
               auth=interactive_auth)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code FH8DE6L2F to authenticate.
Interactive authentication successfully completed.


In [41]:
model = Model.register(model_path=model_path, 
                       model_name=model_name, 
                       tags={"data": "german_credit_risk", "model": "classification"},
                       description="credit risk sample scikit model from Watson Studio",
                       workspace=ws)

Registering model german_credit_risk


## Deploy model as web service

To build the correct environment for ACI, provide the following:

 - A scoring script to show how to use the model
 - An environment file to show what packages need to be installed
 - A configuration file to build the ACI
 - The model you trained before

### Create scoring script

Create the scoring script, called `score.py`, used by the web service call to show how to use the model.

You must include two required functions into the scoring script:
 - The `init()` function, which typically loads the model into a global object. This function is run only once when the Docker container is started. 

 - The `run(input_data)` function uses the model to predict a value based on the input data. Inputs and outputs to the run typically use JSON for serialization and de-serialization, but other formats are supported.

In [53]:
%%writefile score_azure.py
import json
import numpy as np
import os
import pandas as pd
from sklearn.externals import joblib
from sklearn.linear_model import LogisticRegression
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path('german_credit_risk')
    model = joblib.load(model_path)

def run(input_data):
    try:
        if type(input_data) is str:
            dict_data = json.loads(input_data)
        else:
            dict_data = input_data
            
        data = pd.DataFrame.from_dict(dict_data['input'])   
        predictions = model['postprocessing'](model['model'].predict(data))
        scores = model['model'].predict_proba(data).tolist()
        records = []
        
        for pred, prob in zip(predictions, scores):
            records.append({"Scored Labels": pred, "Scored Probabilities": prob})
 
        result = {'output': records}
        
        return json.dumps(result)
    except Exception as e:
        result = str(e)
        # return error message back to the client
        return json.dumps({"error": result})

Writing score_azure.py


### Create environment file

Create an environment file, called `myenv.yml`, that specifies all of the script's package dependencies. 

This file is used to ensure that all of those dependencies are installed in the Docker image.

In [54]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_conda_package("scikit-learn==0.20.2")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())

Review the content of the `myenv.yml` file.

In [55]:
with open("myenv.yml","r") as f:
    print(f.read())

# Conda environment specification. The dependencies defined in this file will
# be automatically provisioned for runs with userManagedDependencies=False.

# Details about the Conda environment file format:
# https://conda.io/docs/user-guide/tasks/manage-environments.html#create-env-file-manually

name: project_environment
dependencies:
  # The python interpreter version.
  # Currently Azure ML only supports 3.5.2 and later.
- python=3.6.2

- pip:
    # Required packages for AzureML execution, history, and data preparation.
  - azureml-defaults

- scikit-learn==0.20.2
channels:
- conda-forge



### Create configuration file

Create a deployment configuration file and specify the number of CPUs and gigabyte of RAM needed for your ACI container.

In [56]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=0.5, 
                                               memory_gb=0.5, 
                                               tags={"data": "credit german",  "method" : "sklearn"}, 
                                               description='Predict Credit Risk with sklearn from Watson Studio')

### Deploy in ACI

Configure the image and deploy it with following steps:

 - Build an image
 - Register that image under the workspace
 - Send the image to the ACI container
 - Start up a container in ACI using the image
 - Get the web service HTTP endpoint

**Note:** Estimated time to complete: **about 7-8 minutes**.

In [57]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

image_config = ContainerImage.image_configuration(execution_script="score_azure.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service_az = Webservice.deploy_from_model(workspace=ws,
                                          name='german-credit-risk-rohit',
                                          deployment_config=aciconfig,
                                          models=[model],
                                          image_config=image_config)

service_az.wait_for_deployment(show_output=True)

Creating image
Running...............................
Succeeded
Image creation operation finished for image german-credit-risk-rohit:3, operation "Succeeded"
Creating service
Running................
SucceededACI service creation operation finished, operation "Succeeded"
CPU times: user 580 ms, sys: 44 ms, total: 624 ms
Wall time: 4min 8s


In [75]:
print("POST scoring request to url: ", service_az.scoring_uri)

POST scoring request to url:  http://839a1e55-422e-4d25-b636-3427dc55cded.westus.azurecontainer.io/score


## Test deployed model

The following code goes through these steps:

 - Prepare scoring payload
 - Send scoring request using an HTTP request and print the returned predictions

In [89]:
import requests

### Prepare scoring payload

In [66]:
scorig_data = {"input":[
                {
                "CheckingStatus": "0_to_200", "LoanDuration": 31, "CreditHistory": "credits_paid_to_date", "LoanPurpose": "other",
                "LoanAmount": 1889, "ExistingSavings": "100_to_500", "EmploymentDuration": "less_1", "InstallmentPercent": 3, "Sex": "female",
                "OthersOnLoan": "none", "CurrentResidenceDuration": 3, "OwnsProperty": "savings_insurance", "Age": 32, "InstallmentPlans": "none",
                "Housing": "own", "ExistingCreditsCount": 1, "Job": "skilled", "Dependents": 1, "Telephone": "none", "ForeignWorker": "yes",
                },
                {
                "CheckingStatus": "no_checking", "LoanDuration": 13, "CreditHistory": "credits_paid_to_date", "LoanPurpose": "car_new",
                "LoanAmount": 1389, "ExistingSavings": "100_to_500", "EmploymentDuration": "1_to_4", "InstallmentPercent": 2, "Sex": "male",
                "OthersOnLoan": "none", "CurrentResidenceDuration": 3, "OwnsProperty": "savings_insurance", "Age": 25, "InstallmentPlans": "none",
                "Housing": "own", "ExistingCreditsCount": 2, "Job": "skilled", "Dependents": 2, "Telephone": "none", "ForeignWorker": "yes",
                }]}

In [94]:
headers = {"Content-Type": "application/json"}
resp = requests.post(service_az.scoring_uri, json=scorig_data, headers=headers)

json.loads(resp.json())

{'output': [{'Scored Labels': 'No Risk',
   'Scored Probabilities': [0.8922524675865824, 0.10774753241341757]},
  {'Scored Labels': 'No Risk',
   'Scored Probabilities': [0.8335192848546905, 0.1664807151453095]}]}